# 1. 머신러닝 파이프라인
## 머신러닝 파이프라인이란?
넓은 의미에서 머신러닝 파이프라인은 데이터 수집, 전처리, 모델 학습, 모델 배포, 예측 등을 순차적으로 처리하는 일련의 프로세스 입니다.

좁은 의미에서 머신러닝 파이프라인은 새로운 데이터 라벨을 예측하는 일련의 프로세스 입니다.

![nn](images/ml_pipeline.png)

## Pipeline 클래스
사이킷런의 Pipeline 클래스를 사용하면 손쉽게 파이프 라인을 학습하고 활용할 수 있습니다.

steps
- 각 요소가 튜블인 리스트로, 이 튜플은 인스턴스의 이름과 인스턴스로 구성됨.
- steps에 있는 인스턴스가 새로 입력된 샘플을 전처리하거나 예측한 결과를 다음 인스턴스로 전달함

steps = [("imputer", imputer), ("scaler", scaler), ("model", model)]

![nn](images/ml_pipeline2.png)

## 예제 데이터 불러오기
파이프 라인을 학습하고 활용하는 방법을 자세히 알아보는데 사용할 데이터를 불러옵니다.

In [1]:
# 데이터 불러오기 예제
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('../../data/Classification/bands.csv')
X = df.drop('y', axis=1)
y = df['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 2022)

In [2]:
df.isnull().sum()

x1     54
x2      5
x3     27
x4      2
x5      1
x6     30
x7     63
x8     55
x9     10
x10    55
x11    55
x12    56
x13    54
x14     6
x15     7
x16    54
x17     7
x18     7
x19     3
y       0
dtype: int64

## 파이프라인 구성 요소 정의
파이프라인의 구성 요소를 정의 하겠습니다

In [3]:
# 파이프라인의 구성 요소를 정의하겠습니다.
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC

imputer = SimpleImputer(strategy='mean')
scaler = MinMaxScaler()
model = SVC(kernel='rbf')

파이프라인의 구성 요소는 학습도 동시에 하므로 puter, scaler, model 모두 학습하지 않았음


## 파이프라인 정의 및 학습
파이프라인을 정의하고 학습해보겠습니다.

In [4]:
# 파이프라인 정의 및 학습 예제
from sklearn.pipeline import Pipeline
P = Pipeline([('imputer', imputer), ('scaler', scaler), ('model', model)])
P.fit(X_train, y_train)

Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', MinMaxScaler()),
                ('model', SVC())])

In [5]:
display(P.predict(X_test))

array([0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1,
       1, 1, 1], dtype=int64)

In [6]:
P = Pipeline([("imputer", imputer), ("scaler", scaler)])
P.fit(X_train, y_train)
display(P.transform(X_test)[0])

array([0.31578947, 0.13513514, 0.4175    , 0.39090909, 0.375     ,
       0.52919021, 0.16666667, 0.15794469, 0.43137255, 0.40924376,
       0.5298976 , 0.10813766, 0.00585399, 0.83333333, 0.35714286,
       0.3036564 , 0.66666667, 0.57058481, 1.        ])

- 위 결과는 결측이 평균으로 대체되고 스케일링이 된 X_test가 SVC 모델에 입력돼 나온 결과임
- 즉, predict 메서드는 파이프라인의 마지막 요소인 SVC 모델의 메서드를 사용한 것임

## Pipeline 클래스의 단점
pipeline 클래스는 손쉽게 파이프라인을 구축할 수 있다는 장점이 있지만, 몇몇 문제가 있어 실무에서 활용하기 어렵습니다.
- 사이킷런의 인스턴스가 아닌 다른 클래스의 인스턴스가 파이프라인에 포함되면 정상적으로 작동하지 않을 수 있음
- 하이퍼 파라미터 튜닝 등을 할 때 전체 파이프라인을 계속해서 수정해야 해서 번거로움
- 특히, 이미 학습한 인스턴스를 여러 번 재학습해야 할 수 있음

## 파이프라인 커스터마이징
파이프라인의 각 요소를 독립적으로 학습하고 함수를 사용해 연결하는 방식으로 파이프라인을 구현 합니다.

In [7]:
# 파이프라인 커스터마이징 예제

# imputer.fit(X_train, y_train) → Error 발생.... why??
# scaler.fit(X_train, y_train)
# model.fit(X_train, y_train)

X_train = imputer.fit_transform(X_train)
X_train = scaler.fit_transform(X_train)
model.fit(X_train, y_train)

SVC()

In [8]:
def my_pipeline(X, imputer, scaler, model):
    X = imputer.transform(X)
    X = scaler.transform(X)
    pred_Y = model.predict(X)
    return pred_Y

In [9]:
pred_Y = my_pipeline(X_test, imputer, scaler, model)
display(pred_Y[:5])

array([0, 1, 0, 1, 0], dtype=int64)

# 2. 피클을 활용한 모델 저장 및 불러오기
## 피클 모듈
피클 모듈은 파이썬 객체를 저장하고 불러오는데 사용합니다. 새로운 데이터가 입력될 때마다 모델을 재학습할 수 없으므로, 학습했던 파이프라인 혹은 인스턴스를 피클을 사용해 저장했다가 필요하면 꺼내써야 합니다.

주요함수

dump
- 파이썬 객체를 파일에 저장 합니다.
- obj : 저장할 인스턴스
- file : 인스턴스를 저장할 파일 (주로 pckl 혹은 pkl 확장자를 가진 바이너리 파일)

load
- 저장한 인스턴스를 볼러옵니다. 저장한 인스턴스가 둘 이상이라면 저장한 순서대로 불러옵니다.
- file : 인스턴스가 저장된 파일

## 피클 모듈을 활용한 파이프라인 저장 및 불러오기
피클 모듈을 활용해서 파이프라인을 저장하고 불러오겠습니다

In [10]:
# 파이프라인 저장하기 예제
import pickle

with open('my_pipeline.pckl', 'wb') as f:
    pickle.dump(imputer, f)
    pickle.dump(scaler, f)
    pickle.dump(model, f)

- 라인 4: with 구문을 이용해 my_pipeline.pckl을 바이너리로 쓰기 모드인 "wb"로 설정해서 f로 엽니다.
- 라인 5 – 7: imputer, scaler, model을 순서대로 f에 저장합니다. 

## 사전으로 변환하여 저장하기
파이프라인이 길어지면 인스턴스를 저장한 순서를 기억해야 하는 문제가 있습니다.

사전으로 병합하여 저장하는 것을 추천

In [11]:
my_pipeline_dict = {'imputer' : imputer, 'scaler' : scaler, 'model' : model}

with open('my_pipeline_dict.pckl', 'wb') as f:
    pickle.dump(my_pipeline_dict, f)